In [56]:
import pandas as pd
import datetime

In [59]:
class SOT(object):
    def __init__(self, sot_file):
        self.sot_df = pd.read_csv(sot_file)
        self.sot_df['Date'] = pd.to_datetime(self.sot_df['Date'], dayfirst=True) 
        
    def __call__(self):
        return self.sot_df
        
    def save_csv(self):
        print(datetime.datetime.now().strftime("%Y-%m-%d-%H:%M"))
    
    def update_budget(self, budget_records):
        return budget_records

    def prep_second_bank(self, csv_location):
        second_bank = pd.read_csv(csv_location)
        second_bank['Date'] = pd.to_datetime(second_bank['Date'], dayfirst=True)
        second_bank.columns = [col.strip() for col in second_bank]
        return second_bank.sort_index()
    
    def update_bank(self, second_bank_csv):
        second_bank_df = self.prep_second_bank(second_bank_csv)
        updated_sot = self.sot_df
        for n, row in second_bank_df.iterrows():
            Date, Type, Description, Value, Balance, Account_name, Account_Number = row # *Something tat requires configing
            key = {'Value':Value, 'Date':Date}
            row_type = self.bank_row_type(key, second_bank_df, updated_sot)
            updated_sot = self.bank_row_type_strategy(row_type, row, second_bank_df, updated_sot)
        return updated_sot

    def bank_row_type(self, key, bank_df, updated_sot):
        match_bank_cols = (updated_sot['Value'] == key['Value'])# & (updated_sot['Date'].isin(key['Date']) # *Something tat requires configing
        matched_bank = match_bank_cols.sum()
        #assert matched_bank < 2, 'multiple match for bank - check function to match bank with bank'

        if not matched_bank:
            # New bank entry...
            matched_budget = (updated_sot['budget_date'] == key)
            number_budget_matches = matched_budget.sum()
            if number_budget_matches:
                # Matches 
                if number_budget_matches == 1:
                    # found unique match
                    match = updated_sot[matched_budget].index
                    assert len(match) == 1, 'match is not unique'
                    return 'unique match', match[0]
                elif number_budget_matches > 1:
                    # found multiple matches
                    return 'multiple match'
            else:
                print('****',key)
                # Need to make new row
                return 'brand new'
        else:
            # Entry already found in bank section, no need to do anything
            return 'existing bank match'
        
    def bank_row_type_strategy(self, row_type, row, second_bank_df, updated_sot):
        bank_cols = second_bank_df.columns
        if row_type == 'existing bank match':
            return updated_sot  # Do nothing
        elif row_type == 'brand new':
            new_row = pd.DataFrame(data=row, index=bank_cols).transpose()
            return pd.concat([updated_sot, new_row])  # Add new row to updated_df
        elif row_type[0] == 'unique match':
            correct_row = updated_sot
            for i in range(len(row)):
                updated_sot = updated_sot.set_value(row_type[1], bank_cols[i],row[i])
            return updated_sot  # Match with 
        elif row_type == 'multiple match':
            print('multiple match: {}'.format(row))
            return updated_sot  # Do nothing
        else:
            raise Exception('unhandled case of bank row type')
                
    


sot = SOT('first_SOT.csv')


In [60]:
x = sot.update_bank('25.11-25.12 -- readied.csv')
x.to_csv('did_it_work.csv')
x 

**** {'Value': -4.45, 'Date': Timestamp('2015-12-23 00:00:00')}
**** {'Value': -53.0, 'Date': Timestamp('2015-12-24 00:00:00')}


,Account,Account Name,Account Number,Balance,Category,Date,Description,Expense Amount,Notes,Payee,SubCategory,Type,Unnamed: 0,Value,budget_date
0,NaN,'Current,'162632-10651269,1009.03,NaN,2015-08-20 00:00:00,'LBTH COUNCIL TAX,NaN,NaN,NaN,NaN,D/D,0,-98,NaN
1,NaN,'Current,'162632-10651269,982.03,NaN,2015-08-21 00:00:00,"'9285 18AUG15 , DELIVEROO.CO.UK , 02033223444 GB",NaN,NaN,NaN,NaN,POS,1,-27,NaN
2,NaN,'Current,'162632-10651269,962.03,NaN,2015-08-21 00:00:00,'NATWEST 20AUG,NaN,NaN,NaN,NaN,C/L,2,-20,NaN
3,NaN,'Current,'162632-10651269,960.04,NaN,2015-08-24 00:00:00,"'9285 21AUG15 , AMAZON DIGITAL , DWNLDS , 866-...",NaN,NaN,NaN,NaN,POS,3,-1.99,NaN
4,NaN,'Current,'162632-10651269,942.84,NaN,2015-08-24 00:00:00,"'9285 23AUG15 , DELIVERANCE , LIMITED , LONDON GB",NaN,NaN,NaN,NaN,POS,4,-17.2,NaN
5,NaN,'Current,'162632-10651269,812.15,NaN,2015-08-26 00:00:00,"'9285 25AUG15 , UDACITY, INC. , 8778877815 US ...",NaN,NaN,NaN,NaN,POS,5,-130.69,NaN
6,NaN,'Current,'162632-10651269,748.83,NaN,2015-08-26 00:00:00,"'9285 25AUG15 , MONSTER SUPPLEMENT, HULL GB",NaN,NaN,NaN,NaN,POS,6,-63.32,NaN
7,NaN,'Current,'162632-10651269,730.38,NaN,2015-08-26 00:00:00,'PNET2674351-1,NaN,NaN,NaN,NaN,D/D,7,-18.45,NaN
8,NaN,'Current,'162632-10651269,1356.77,NaN,2015-08-27 00:00:00,"'CONTRACTOR UMBRELL, CONTRACTOR UMBRELL, FP 27...",NaN,NaN,NaN,NaN,BAC,8,626.39,NaN
9,NaN,'Current,'162632-10651269,1333.77,NaN,2015-08-27 00:00:00,"'9285 26AUG15 , DELIVERANCE , LIMITED , LONDON GB",NaN,NaN,NaN,NaN,POS,9,-23,NaN
